# CoCa (Contrastive Captioners are Image-Text Foundation Models)  
- paper review

## Introduction  
- 그동안 foundation model에 대한 연구가 활발히 진행  
- 크게 single-encoder, dual-encoder, encoder-decoder 3가지 architecture로 이루어짐  
- 1. single-encoder  
    - ImageNet과 같은 dataset을 가지고 image classification을 수행하며 cross entropy loss로 최적화  
    - 그러나 이는 labeled dataset에 크게 의존하며 vision and language modalities에 적용하는 downstream task 적용이 불가능  
- 2. dual-encoder  
    - image-text foundation model을 위해 dual-encoder architecture가 연구됨  
    - 그러나 이러한 모델들은 VQA (visual question answering)과 같이 vision-language understanding task에는 적용이 불가능  
- 3. encoder-decoder  
    - generic vision과 multimodal representation을 학습하기 위해 제안됨  
    - 그러나 이들은 image embedding에 해당하는 text representation을 생성하지 못해 crossmodel alignment task에 적합하지 않음  


<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/figure1.png?raw=true" width=700>

  
  
- 그래서 이 논문에서는 위의 3가지 architecture 유형들을 모두 통합하는 image-text foundation 모델을 제안  
- 이를 Contrastive Captioners, 즉 CoCa라고 명명  
- 저자들은 figure1처럼 architecture를 제안했으며 unimodal decoder에서는 cross-attention을 생략해 text-only representation을 encoding하도록 했다  
- 또한 multimodal decoder에만 cross-attention을 적용해 multimodal image-text representation을 학습하도록 설계함

## Approach  
### Natural Language Supervision  
#### Single-Encoder Classification  
- classic single-encoder는 ImageNet, JFT와 같은 dataset을 classification task에 적용하는 것이다  
- VGG, ResNet, ViT 등 일반적으로 아주 잘 알고 있는 모델들이 있다  
$$\mathcal{L}_{cls}=-p(y)\log q_{\theta}(x)$$  

- loss function은 위의 cross entropy와 같으며 이를 최적화하는 문제로 귀결된다  

#### Dual-Encoder Contrastive Learning  
- single-encoder는 human-annotated labels과 data cleaning이 필요하다  
- dual-encoder approach는 noisy web-scale text description을 활용하고 free-form text를 encoding하기 위한 learnable text tower를 도입  
- 그래서 이 architecture는 다음과 같은 loss function을 통해 최적화한다  
$$\mathcal{L}_{con}=-\frac{1}{N}\left(\sum_i^N\log\frac{exp\left(x_i^Ty_i/\sigma\right)}{\sum_{j=1}^Nexp\left(x_i^Ty_j/\sigma\right)} + \sum_i^N\log\frac{exp\left(y_i^Tx_i/\sigma\right)}{\sum_{j=1}^N exp\left(y_i^Tx_j/\sigma\right)}\right)$$  

- 여기서 $x_i$와 $y_j$는 i번째 pair에 있는 image, j번째 pair에 있는 text의 normalized embedding에 해당한다  
- 즉, contrastive learning에 쓰이는 contrastive loss function과 유사하다  
- positive와 positive의 pair는 최대한 가깝게, negative pair는 최대한 멀리 있도록 최적화한다  
- 이를 image-to-text, text-to-image 각 2개의 term으로 구분해놓았다

#### Encoder-Decoder Captioning  
- dual-encoder는 text 전체를 encoding하는 반면, captioner와 같은 generative approach는 detailed granularity에 집중하고 $y$의 정확한 tokenized text를 예측해야 한다  
- 일반적으로 encoder-decoder architecture에서 image encoder는 latent encoded feature를 출력하고 text decoder는 paired text $y$의 conditional likelihood를 최대화하도록 학습한다  

$$\mathcal{L}_{cap}=-\sum_{t=1}^T\log P_{\theta}\left(y_t\vert y_{<t}, x\right)$$  

- 위 2개의 architecture와 다르게 captioning은 image-test representation을 학습한다

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/figure2.png?raw=true" width=700>  
  

### Contrastive Captioners Pretraining  
- 위의 figure2에서 CoCa의 architecture를 보여주고 있다  
- Introduction에서 소개했던대로 unimodal text representation을 encoding하기 위한 decoder layers의 first half는 cross-attention을 생략하고 multimodal image-text representation을 encoding하기 위한 나머지 decoder layers에는 cross-attention을 적용했다  
- 그래서 CoCa decoder는 unimodal과 multimodal text representation을 동시에 생성할 수 있다  
- contrastive and generative objective는 다음과 같다  
$$L_{CoCa}=\lambda_{Con}\cdot \mathcal{L}_{Con} + \lambda_{Cap}\cdot \mathcal{L}_{Cap}$$  

#### Decoupled Text Decoder and CoCa Architecture  
- captioning approach는 text의 conditional likelihood를 최적화하는 반면 contrastive approach는 unconditional text representation을 활용  
- 이러한 딜레마를 해결하고 두 methods를 합치기 위해 decoupled decoder design을 제안  
- 즉, unimodal과 multimodal decoder로 분리하고 unimodal decoder에는 cross-attention을 생략  
- 모든 decoder layers는 future tokens에 대해 attention하지 못하도록 설정  
- contrastive loss $\mathcal{L}_{Con}$에 대해서는, learnable cls token을 input sentence 마지막에 추가하고 이의 output을 unimodal decoder의 text embedding으로 사용  
- unimodal, multimodal decoder layers의 갯수는 $n_{uni}=n_{multi}$로 맞춘다  
- $288\times 288$ resolution, $18\times 18$ patch size 즉 256 image tokens로 pretrain을 수행  
- layers의 갯수을 조절해 base, large 등의 모델 사이즈를 결정  

#### Attentional Poolers  
- preliminary experiments에서 single pooled image embedding은 global representation으로써 visual recognition task에 도움을 주는 반면, 더 많은 visual tokens은 region-level features를 필요로 하는 multimodal understanding task에 도움이 된다  
- 따라서 CoCa는 task-specific attentional pooling을 적용하며 다양한 타입의 training objectives 그리고 downstream tasks에 적용하고자 한다  


## Experiments  
### Training Setup  
#### Data
- JFT-3B dataset with label names as the paired texts  
- ALIGN dataset with noisy alt-texts  

#### Optimization  
- batch size of 65536 image-text pairs  
- 500k steps, 5 epochs on JFT and 10 epochs on ALIGN  
- $\lambda_{Cap}=2.0$ and $\lambda_{Con}=1.0$  
- temperature $\tau=0.07$   
- Adafactor optimizer with $\beta_1=0.9$ and $\beta_2=0.999$  
- decoupled weight decay ratio of 0.01  
- warm up the learning rate for the first 2% of training steps to a peak value of $8 \times 10^{-4}$ with linearly decay  

### Main Results  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/figure4.png?raw=true" width=700>  
  
- 위의 figure4에서 CoCa와 다른 foundation models의 성능을 비교한 것  
- CoCa가 압도적으로 좋은 성능을 보여줌  

#### Visual Recognition Tasks  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/table2.png?raw=true" width=700>  
  
  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/figure5.png?raw=true" width=700>


#### Crossmodal Alignment Tasks  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/table3.png?raw=true" width=700>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/table4.png?raw=true" width=700>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/table5.png?raw=true" width=700>  


#### Image Captioning and Multimodal Understanding Tasks  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/table6.png?raw=true" width=700>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/table7.png?raw=true" width=700>  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/figure6.png?raw=true" width=700>  


### Ablation Analysis  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/CoCa/figures/table8.png?raw=true" width=700>  

## Conclusion   
- 이 논문에서 image-text foundation model CoCa를 제안  
- contrastive와 captioning objectives를 효율적으로 결합해 encoder-decoder model을 최적화  
- 그 결과 다양한 downstream tasks에서 SOTA를 달성 